# Notebook 02: Mechanistic Probes

**Research Question:** Can we detect ground truth action-taking from model activations?

This notebook:
1. Extracts activations at multiple positions and layers
2. Trains reality (tool_used) and narrative (claims_action) probes
3. **Critical:** Position analysis (first_assistant vs before_tool)
4. Analyzes probe behavior on fake action cases

**Key hypothesis:** If `first_assistant` accuracy > 80%, probe is detecting action-grounding, not just tool syntax.

## Setup

In [ ]:
import sys
sys.path.append('..')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from src.utils.logging import setup_logging
from src.config import get_config
from src.data.io import load_episodes, save_activations, load_activations
from src.extraction import extract_activations_batch
from src.analysis.probes import train_and_evaluate, analyze_probe_on_category, save_probe
from src.analysis.visualization import (
    plot_confusion_matrix,
    plot_roc_curve,
    plot_position_accuracy,
    plot_layer_analysis,
)
from src.analysis.statistics import compute_roc_auc, bootstrap_metrics

setup_logging(level="INFO")
config = get_config()

print(f"Extraction config:")
print(f"  Positions: {config.extraction.positions}")
print(f"  Layers: {config.extraction.layers}")

## 1. Load Episodes

In [ ]:
# Load episodes from Phase 1
episodes_collection = load_episodes(config.data.processed_dir / "episodes.parquet")
episodes = episodes_collection.episodes

print(f"Loaded {len(episodes)} episodes")
print(f"\nCategory breakdown:")
summary = episodes_collection.summary()
for cat, count in summary['categories'].items():
    print(f"  {cat}: {count}")

## 2. Extract Activations

Extract at 3 positions × 5 layers = 15 samples per episode.

**WARNING:** This takes 1-2 hours on GPU depending on model size.

In [ ]:
# Extract activations
dataset = extract_activations_batch(
    episodes=episodes,
    positions=config.extraction.positions,
    layers=config.extraction.layers,
    model_id=config.model.id,
    save_path=config.data.processed_dir / "activations.parquet",
    verbose=True,
)

print(f"\nExtracted {len(dataset)} activation samples")
print(f"Activation shape: {dataset.activations.shape}")
print(f"Hidden size: {dataset.hidden_size}")

In [ ]:
# Load activations (if already extracted)
# dataset = load_activations(config.data.processed_dir / "activations.parquet")
# print(f"Loaded {len(dataset)} activation samples")

In [ ]:
# Summary
summary = dataset.summary()
print("\nDataset summary:")
print(f"  Samples: {summary['n_samples']}")
print(f"  Positions: {summary['positions']}")
print(f"  Layers: {summary['layers']}")
print(f"  Categories: {summary['categories']}")

## 3. Train Probes

Train two probes:
- **Reality probe:** Predicts `tool_used` (ground truth)
- **Narrative probe:** Predicts `claims_action` (model's claim)

In [ ]:
# Filter to single position and layer for initial probe
# Use mid_response at layer 16 (middle layer)
probe_dataset = dataset.filter_by_position("mid_response").filter_by_layer(16)

print(f"Probe training dataset: {len(probe_dataset)} samples")

In [ ]:
# Train reality probe
reality_probe, reality_train_metrics, reality_test_metrics = train_and_evaluate(
    probe_dataset,
    label_type="reality",
    test_size=0.2,
    random_state=42,
)

print("\nReality Probe (predicts tool_used):")
print("Train Metrics:")
print(reality_train_metrics)
print("\nTest Metrics:")
print(reality_test_metrics)

In [ ]:
# Train narrative probe
narrative_probe, narrative_train_metrics, narrative_test_metrics = train_and_evaluate(
    probe_dataset,
    label_type="narrative",
    test_size=0.2,
    random_state=42,
)

print("\nNarrative Probe (predicts claims_action):")
print("Train Metrics:")
print(narrative_train_metrics)
print("\nTest Metrics:")
print(narrative_test_metrics)

In [ ]:
# Save probes
save_probe(reality_probe, config.data.processed_dir / "reality_probe.pkl")
save_probe(narrative_probe, config.data.processed_dir / "narrative_probe.pkl")

print("Probes saved.")

## 4. CRITICAL: Position Analysis

**Key test:** Does the probe work at `first_assistant` (before any tool tokens)?

If yes → probe detects action-grounding, not syntax  
If no → probe might just be detecting `<<CALL` tokens

In [ ]:
# Train probes at each position (using layer 16)
position_accuracies = {}

for position in config.extraction.positions:
    print(f"\nTraining probe at position: {position}")
    
    pos_dataset = dataset.filter_by_position(position).filter_by_layer(16)
    
    if len(pos_dataset) == 0:
        print(f"  No samples found for {position}")
        continue
    
    probe, _, test_metrics = train_and_evaluate(
        pos_dataset,
        label_type="reality",
        random_state=42,
    )
    
    position_accuracies[position] = test_metrics.accuracy
    print(f"  Test accuracy: {test_metrics.accuracy:.1%}")

print("\nPosition accuracies:")
for pos, acc in position_accuracies.items():
    status = "✓ PASS" if acc > 0.80 else "✗ FAIL"
    print(f"  {pos}: {acc:.1%} {status}")

## 9. Investigating before_tool Underperformance

**Problem:** The `before_tool` position shows lower accuracy (78.9%) than `first_assistant` (83.9%), which is counterintuitive.

**Hypotheses:**
- **A:** Position extraction is ill-defined for no-tool cases
- **B:** Representation shifts from "decision" to "execution" 
- **C:** Token-level noise at before_tool position


In [ ]:
# Visualize position analysis
fig = plot_position_accuracy(
    position_accuracies,
    title="Reality Probe Accuracy by Token Position",
    save_path=config.data.figures_dir / "figure2_position_accuracy",
)

plt.show()

# Check critical result
first_assistant_acc = position_accuracies.get('first_assistant', 0)
if first_assistant_acc > 0.80:
    print(f"\n✓ CRITICAL RESULT: first_assistant accuracy = {first_assistant_acc:.1%} > 80%")
    print("  → Probe detects action-grounding, not just syntax!")
else:
    print(f"\n✗ WARNING: first_assistant accuracy = {first_assistant_acc:.1%} < 80%")
    print("  → May be detecting syntax, not action-grounding")

uz

### 9.1 Position Extraction Diagnostics

Check what positions are actually extracted for different cases, especially for `before_tool` when `tool_used=False`.


In [ ]:
# Check position extraction for different cases
from collections import Counter

# Count samples by position and tool_used
position_tool_counts = {}
for sample in dataset.samples:
    key = (sample.position, sample.tool_used)
    position_tool_counts[key] = position_tool_counts.get(key, 0) + 1

print("Sample counts by position and tool_used:")
for (pos, tool_used), count in sorted(position_tool_counts.items()):
    tool_used_str = str(tool_used)
    print(f"  {pos:20s} | tool_used={tool_used_str:5s} | n={count:4d}")

# Check specifically for before_tool
before_tool_samples = [s for s in dataset.samples if s.position == "before_tool"]
print(f"\nbefore_tool samples: {len(before_tool_samples)}")
print(f"  tool_used=True:  {sum(1 for s in before_tool_samples if s.tool_used)}")
print(f"  tool_used=False: {sum(1 for s in before_tool_samples if not s.tool_used)}")

# Check if before_tool samples have valid token_str
valid_before_tool = [s for s in before_tool_samples if s.token_str is not None]
print(f"\nbefore_tool samples with token_str: {len(valid_before_tool)}/{len(before_tool_samples)}")

# Sample some token strings
if valid_before_tool:
    print("\nSample tokens at before_tool position:")
    for i, sample in enumerate(valid_before_tool[:10]):
        token_repr = repr(sample.token_str) if sample.token_str else "None"
        print(f"  {i+1}. tool_used={sample.tool_used}, token={token_repr[:50]}")
    
    # Print actual episodes for problematic cases (tool_used=False but token='<<')
    print("\n" + "="*60)
    print("Investigating problematic cases: tool_used=False but token='<<'")
    print("="*60)
    
    problematic_samples = [s for s in valid_before_tool if not s.tool_used and s.token_str and '<<' in s.token_str]
    print(f"\nFound {len(problematic_samples)} samples with tool_used=False but token contains '<<'")
    
    if problematic_samples:
        print("\nExample episodes (showing first 3):")
        for i, sample in enumerate(problematic_samples[:3]):
            # Find the corresponding episode
            episode = next((ep for ep in episodes if ep.id == sample.episode_id), None)
            
            if episode:
                print(f"\n{'='*60}")
                print(f"Example {i+1}: Episode ID = {sample.episode_id}")
                print(f"  Position: {sample.position}, Layer: {sample.layer}")
                print(f"  Token at before_tool: {repr(sample.token_str)}")
                print(f"  tool_used: {sample.tool_used}")
                print(f"  claims_action: {sample.claims_action}")
                print(f"  Category: {sample.category}")
                print(f"\n  Episode details:")
                print(f"    System variant: {episode.system_variant}")
                print(f"    Social pressure: {episode.social_pressure}")
                print(f"    Tool type: {episode.tool_type}")
                
                # Show assistant reply (truncated)
                if hasattr(episode, 'assistant_reply'):
                    reply_preview = episode.assistant_reply[:500] if len(episode.assistant_reply) > 500 else episode.assistant_reply
                    print(f"\n  Assistant reply (first 500 chars):")
                    print(f"    {reply_preview}")
                    if len(episode.assistant_reply) > 500:
                        print(f"    ... (truncated, total length: {len(episode.assistant_reply)})")
                
                # Check if there's tool call syntax in the reply
                if hasattr(episode, 'assistant_reply'):
                    has_call = '<<CALL' in episode.assistant_reply
                    has_end = '>>' in episode.assistant_reply
                    print(f"\n  Tool call syntax check:")
                    print(f"    Contains '<<CALL': {has_call}")
                    print(f"    Contains '>>': {has_end}")
                    
                    # Extract tool call portion if present
                    if '<<CALL' in episode.assistant_reply:
                        call_start = episode.assistant_reply.find('<<CALL')
                        call_end = episode.assistant_reply.find('>>', call_start)
                        if call_end > call_start:
                            call_text = episode.assistant_reply[call_start:call_end+2]
                            print(f"    Tool call text: {call_text[:200]}")
                        else:
                            print(f"    Tool call text (incomplete?): {episode.assistant_reply[call_start:call_start+200]}")
                
                # Check tool_call_raw field (if available)
                if hasattr(episode, 'tool_call_raw') and episode.tool_call_raw:
                    print(f"\n  tool_call_raw field:")
                    print(f"    {episode.tool_call_raw[:300]}")
                elif hasattr(episode, 'tool_call_raw'):
                    print(f"\n  tool_call_raw: None (tool was not executed)")
                
                # Show tool_call_args if available
                if hasattr(episode, 'tool_call_args') and episode.tool_call_args:
                    print(f"\n  tool_call_args:")
                    print(f"    {episode.tool_call_args}")
            else:
                print(f"\n  Episode {sample.episode_id} not found in episodes list")


In [ ]:
# Split accuracy by tool_used at each position
from sklearn.metrics import accuracy_score

split_accuracies = {}

for position in config.extraction.positions:
    pos_dataset = dataset.filter_by_position(position).filter_by_layer(16)
    
    if len(pos_dataset) == 0:
        continue
    
    # Split by tool_used
    tool_used_mask = np.array([s.tool_used for s in pos_dataset.samples])
    tool_used_dataset = pos_dataset._filter_by_indices(np.where(tool_used_mask)[0])
    no_tool_dataset = pos_dataset._filter_by_indices(np.where(~tool_used_mask)[0])
    
    print(f"\n{position}:")
    print(f"  Total samples: {len(pos_dataset)}")
    print(f"  tool_used=True: {len(tool_used_dataset)}")
    print(f"  tool_used=False: {len(no_tool_dataset)}")
    
    # Train probe on full dataset
    probe, _, _ = train_and_evaluate(
        pos_dataset,
        label_type="reality",
        random_state=42,
    )
    
    # Evaluate on each subset
    if len(tool_used_dataset) > 0:
        X_tool, y_tool = tool_used_dataset.to_sklearn_format("reality")
        y_pred_tool = probe.predict(X_tool)
        acc_tool = accuracy_score(y_tool, y_pred_tool)
        print(f"  Accuracy on tool_used=True:  {acc_tool:.1%}")
        split_accuracies[(position, True)] = acc_tool
    
    if len(no_tool_dataset) > 0:
        X_no_tool, y_no_tool = no_tool_dataset.to_sklearn_format("reality")
        y_pred_no_tool = probe.predict(X_no_tool)
        acc_no_tool = accuracy_score(y_no_tool, y_pred_no_tool)
        print(f"  Accuracy on tool_used=False: {acc_no_tool:.1%}")
        split_accuracies[(position, False)] = acc_no_tool

# Summary table
print("\n" + "="*60)
print("Summary: Accuracy by Position and tool_used")
print("="*60)
for position in config.extraction.positions:
    acc_true = split_accuracies.get((position, True), None)
    acc_false = split_accuracies.get((position, False), None)
    acc_overall = position_accuracies.get(position, None)
    
    print(f"\n{position}:")
    if acc_true is not None:
        print(f"  tool_used=True:  {acc_true:.1%}")
    if acc_false is not None:
        print(f"  tool_used=False: {acc_false:.1%}")
    if acc_overall is not None:
        print(f"  Overall:         {acc_overall:.1%}")


### 9.3 Probe Direction Comparison Across Positions

Train separate probes at each position and compare their directions using cosine similarity.


In [ ]:
# Train probes at each position and compare directions
from src.analysis.probes import get_probe_direction, compare_probes
from itertools import combinations

position_probes = {}

print("Training probes at each position...")
for position in config.extraction.positions:
    pos_dataset = dataset.filter_by_position(position).filter_by_layer(16)
    
    if len(pos_dataset) == 0:
        continue
    
    probe, _, _ = train_and_evaluate(
        pos_dataset,
        label_type="reality",
        random_state=42,
    )
    
    position_probes[position] = probe
    print(f"  {position}: trained")

# Compare probe directions
print("\n" + "="*60)
print("Probe Direction Similarity Across Positions")
print("="*60)

for pos1, pos2 in combinations(position_probes.keys(), 2):
    comparison = compare_probes(position_probes[pos1], position_probes[pos2], normalize=True)
    cosine_sim = comparison['cosine_similarity']
    
    print(f"\n{pos1:20s} vs {pos2:20s}:")
    print(f"  Cosine similarity: {cosine_sim:.3f}")
    print(f"  L2 distance:       {comparison['l2_distance']:.3f}")
    
    if abs(cosine_sim) > 0.9:
        print("  → Very similar directions")
    elif abs(cosine_sim) > 0.7:
        print("  → Similar directions")
    elif abs(cosine_sim) > 0.5:
        print("  → Partially aligned")
    elif abs(cosine_sim) > 0.3:
        print("  → Different directions")
    else:
        print("  → Very different directions")


## 10. 2D Probe Projection Analysis

**Goal:** Visualize how reality and narrative probes diverge across categories to reveal the fake action mechanism.

The 0.513 cosine similarity suggests partial alignment. We want to see:
- Where fake_action cases fall (should be: low reality, high narrative)
- Where true_action cases fall (should be: high reality, high narrative)
- Where silent_action cases fall (should be: high reality, low narrative)
- Where honest_no_action cases fall (should be: low reality, low narrative)


### 10.1 Project Samples onto Probe Directions

Get test dataset and project activations onto both reality and narrative probe directions.


In [ ]:
# Get test dataset (already split from training)
# Use the same test split as used for probe training
train_dataset, test_dataset = probe_dataset.train_test_split(test_size=0.2, random_state=42)

# Get activations and labels
X_test, y_test_reality = test_dataset.to_sklearn_format("reality")
_, y_test_narrative = test_dataset.to_sklearn_format("narrative")

# Project onto probe directions
# These are logit scores (before sigmoid)
reality_scores = X_test @ reality_probe.coef_.T
narrative_scores = X_test @ narrative_probe.coef_.T

# Convert to probabilities for easier interpretation
reality_probs = reality_probe.predict_proba(X_test)[:, 1]
narrative_probs = narrative_probe.predict_proba(X_test)[:, 1]

# Get category labels
categories = np.array([s.category for s in test_dataset.samples])

print(f"Test dataset: {len(test_dataset)} samples")
print(f"\nCategory breakdown:")
for cat in ['true_action', 'fake_action', 'silent_action', 'honest_no_action', 'wrong_tool']:
    count = np.sum(categories == cat)
    print(f"  {cat}: {count}")

print(f"\nReality scores range: [{reality_scores.min():.2f}, {reality_scores.max():.2f}]")
print(f"Narrative scores range: [{narrative_scores.min():.2f}, {narrative_scores.max():.2f}]")


### 10.2 2D Scatter Plot by Category

Create scatter plot showing where each category falls in probe space.


In [ ]:
# Create 2D scatter plot colored by category
fig, ax = plt.subplots(figsize=(12, 10))

# Color scheme for categories
category_colors = {
    'true_action': 'green',
    'fake_action': 'red',
    'silent_action': 'orange',
    'honest_no_action': 'blue',
    'wrong_tool': 'purple',
}

category_labels = {
    'true_action': 'True Action',
    'fake_action': 'Fake Action',
    'silent_action': 'Silent Action',
    'honest_no_action': 'Honest No Action',
    'wrong_tool': 'Wrong Tool',
}

# Plot each category
for cat in ['true_action', 'fake_action', 'silent_action', 'honest_no_action', 'wrong_tool']:
    mask = categories == cat
    if np.any(mask):
        ax.scatter(
            reality_scores[mask],
            narrative_scores[mask],
            label=category_labels[cat],
            alpha=0.6,
            s=50,
            color=category_colors[cat],
            edgecolors='black',
            linewidths=0.5,
        )

# Add reference lines at 0
ax.axhline(0, c='k', ls='--', alpha=0.3, linewidth=1)
ax.axvline(0, c='k', ls='--', alpha=0.3, linewidth=1)

# Add decision boundaries (logit threshold = 0 corresponds to prob = 0.5)
# For logistic regression, decision boundary is at logit = 0
ax.axhline(0, c='gray', ls=':', alpha=0.5, linewidth=1, label='Decision boundary')
ax.axvline(0, c='gray', ls=':', alpha=0.5, linewidth=1)

# Labels and title
ax.set_xlabel('Reality Probe Score (logit)', fontsize=14)
ax.set_ylabel('Narrative Probe Score (logit)', fontsize=14)
ax.set_title('2D Probe Projection: Reality vs Narrative', fontsize=16, fontweight='bold')
ax.legend(loc='best', fontsize=12)
ax.grid(True, alpha=0.3)

# Add quadrant labels
ax.text(0.02, 0.98, 'Upper Right:\nHigh Reality\nHigh Narrative', 
        transform=ax.transAxes, fontsize=10, verticalalignment='top',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
ax.text(0.02, 0.02, 'Lower Left:\nLow Reality\nLow Narrative', 
        transform=ax.transAxes, fontsize=10, verticalalignment='bottom',
        bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
ax.text(0.98, 0.98, 'Upper Left:\nLow Reality\nHigh Narrative\n(FAKE ACTION)', 
        transform=ax.transAxes, fontsize=10, verticalalignment='top', horizontalalignment='right',
        bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.5))
ax.text(0.98, 0.02, 'Lower Right:\nHigh Reality\nLow Narrative\n(SILENT ACTION)', 
        transform=ax.transAxes, fontsize=10, verticalalignment='bottom', horizontalalignment='right',
        bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))

plt.tight_layout()
plt.savefig(config.data.figures_dir / "figure_probe_2d_projection.png", dpi=300, bbox_inches='tight')
plt.show()

print("Expected pattern:")
print("  - true_action: upper right (high reality, high narrative)")
print("  - fake_action: upper left (low reality, high narrative)")
print("  - silent_action: lower right (high reality, low narrative)")
print("  - honest_no_action: lower left (low reality, low narrative)")
print("  - wrong_tool: high reality, variable narrative")


### 10.3 Disagreement Analysis

Find cases where reality and narrative probes disagree and analyze by category.


In [ ]:
# Get predictions from both probes
reality_pred = reality_probe.predict(X_test)
narrative_pred = narrative_probe.predict(X_test)

# Find disagreements
disagreements = reality_pred != narrative_pred
disagreement_rate = np.mean(disagreements)

print("="*60)
print("Probe Disagreement Analysis")
print("="*60)
print(f"\nOverall disagreement rate: {disagreement_rate:.1%}")
print(f"  Agree: {np.sum(~disagreements)} samples")
print(f"  Disagree: {np.sum(disagreements)} samples")

# Analyze disagreements by category
print("\nDisagreements by category:")
disagreement_by_cat = {}
for cat in ['true_action', 'fake_action', 'silent_action', 'honest_no_action', 'wrong_tool']:
    cat_mask = categories == cat
    if np.any(cat_mask):
        cat_disagreements = disagreements[cat_mask]
        cat_disagreement_rate = np.mean(cat_disagreements)
        cat_total = np.sum(cat_mask)
        cat_disagree_count = np.sum(cat_disagreements)
        disagreement_by_cat[cat] = {
            'rate': cat_disagreement_rate,
            'count': cat_disagree_count,
            'total': cat_total,
        }
        print(f"\n  {cat}:")
        print(f"    Disagreement rate: {cat_disagreement_rate:.1%} ({cat_disagree_count}/{cat_total})")

# Analyze disagreement types
print("\n" + "="*60)
print("Disagreement Types")
print("="*60)

# Reality says 0 (no tool), Narrative says 1 (claims action)
reality_0_narr_1 = (reality_pred == 0) & (narrative_pred == 1)
# Reality says 1 (tool used), Narrative says 0 (no claim)
reality_1_narr_0 = (reality_pred == 1) & (narrative_pred == 0)

print(f"\nReality=0, Narrative=1 (reality says no tool, narrative claims action):")
print(f"  Total: {np.sum(reality_0_narr_1)}")
for cat in ['true_action', 'fake_action', 'silent_action', 'honest_no_action', 'wrong_tool']:
    cat_mask = categories == cat
    count = np.sum(reality_0_narr_1 & cat_mask)
    if count > 0:
        print(f"    {cat}: {count}")

print(f"\nReality=1, Narrative=0 (reality says tool used, narrative doesn't claim):")
print(f"  Total: {np.sum(reality_1_narr_0)}")
for cat in ['true_action', 'fake_action', 'silent_action', 'honest_no_action', 'wrong_tool']:
    cat_mask = categories == cat
    count = np.sum(reality_1_narr_0 & cat_mask)
    if count > 0:
        print(f"    {cat}: {count}")

# Create confusion matrix of probe disagreements
print("\n" + "="*60)
print("Probe Disagreement Confusion Matrix")
print("="*60)
print("\nRows = Reality Probe, Columns = Narrative Probe")
print("         | Claims=0  | Claims=1")
print("---------|----------|----------")
print(f"No Tool  | {np.sum((reality_pred==0) & (narrative_pred==0)):8d} | {np.sum((reality_pred==0) & (narrative_pred==1)):8d}")
print(f"Tool     | {np.sum((reality_pred==1) & (narrative_pred==0)):8d} | {np.sum((reality_pred==1) & (narrative_pred==1)):8d}")

# Cross-tabulation by category
print("\nDisagreement breakdown by category:")
disagreement_crosstab = pd.crosstab(
    categories[disagreements],
    pd.Series([
        'Reality=0,Narr=1' if r == 0 else 'Reality=1,Narr=0'
        for r, n in zip(reality_pred[disagreements], narrative_pred[disagreements])
    ]),
    margins=True
)
print(disagreement_crosstab)


### 10.4 Quantitative Summary

Compute mean scores, separation metrics, and quadrant distribution for each category.


In [ ]:
# Quantitative analysis by category
print("="*60)
print("Quantitative Summary by Category")
print("="*60)

category_stats = {}

for cat in ['true_action', 'fake_action', 'silent_action', 'honest_no_action', 'wrong_tool']:
    mask = categories == cat
    if not np.any(mask):
        continue
    
    cat_reality_scores = reality_scores[mask]
    cat_narrative_scores = narrative_scores[mask]
    cat_reality_probs = reality_probs[mask]
    cat_narrative_probs = narrative_probs[mask]
    
    stats = {
        'n': np.sum(mask),
        'reality_mean_score': np.mean(cat_reality_scores),
        'reality_std_score': np.std(cat_reality_scores),
        'reality_mean_prob': np.mean(cat_reality_probs),
        'narrative_mean_score': np.mean(cat_narrative_scores),
        'narrative_std_score': np.std(cat_narrative_scores),
        'narrative_mean_prob': np.mean(cat_narrative_probs),
    }
    
    category_stats[cat] = stats
    
    print(f"\n{cat}:")
    print(f"  N samples: {stats['n']}")
    print(f"  Reality probe:")
    print(f"    Mean score (logit): {stats['reality_mean_score']:.3f} ± {stats['reality_std_score']:.3f}")
    print(f"    Mean probability:   {stats['reality_mean_prob']:.3f}")
    print(f"  Narrative probe:")
    print(f"    Mean score (logit): {stats['narrative_mean_score']:.3f} ± {stats['narrative_std_score']:.3f}")
    print(f"    Mean probability:   {stats['narrative_mean_prob']:.3f}")

# Compute separation metrics
print("\n" + "="*60)
print("Separation Metrics")
print("="*60)

# Distance between fake_action and true_action clusters
if 'fake_action' in category_stats and 'true_action' in category_stats:
    fake_center = np.array([
        category_stats['fake_action']['reality_mean_score'],
        category_stats['fake_action']['narrative_mean_score']
    ])
    true_center = np.array([
        category_stats['true_action']['reality_mean_score'],
        category_stats['true_action']['narrative_mean_score']
    ])
    distance = np.linalg.norm(fake_center - true_center)
    print(f"\nDistance between fake_action and true_action centers: {distance:.3f}")

# Quadrant distribution
print("\n" + "="*60)
print("Quadrant Distribution")
print("="*60)

# Define quadrants based on logit scores (0 is decision boundary)
quadrants = {
    'Upper Right (High Reality, High Narrative)': (reality_scores > 0) & (narrative_scores > 0),
    'Upper Left (Low Reality, High Narrative)': (reality_scores <= 0) & (narrative_scores > 0),
    'Lower Left (Low Reality, Low Narrative)': (reality_scores <= 0) & (narrative_scores <= 0),
    'Lower Right (High Reality, Low Narrative)': (reality_scores > 0) & (narrative_scores <= 0),
}

print("\nOverall quadrant distribution:")
for quadrant_name, quadrant_mask in quadrants.items():
    count = np.sum(quadrant_mask)
    pct = count / len(categories) * 100
    print(f"  {quadrant_name}: {count:4d} ({pct:5.1f}%)")

print("\nQuadrant distribution by category:")
for cat in ['true_action', 'fake_action', 'silent_action', 'honest_no_action', 'wrong_tool']:
    cat_mask = categories == cat
    if not np.any(cat_mask):
        continue
    
    print(f"\n  {cat}:")
    for quadrant_name, quadrant_mask in quadrants.items():
        count = np.sum(quadrant_mask & cat_mask)
        pct = count / np.sum(cat_mask) * 100 if np.sum(cat_mask) > 0 else 0
        print(f"    {quadrant_name}: {count:4d} ({pct:5.1f}%)")

# Expected vs observed patterns
print("\n" + "="*60)
print("Expected vs Observed Patterns")
print("="*60)
print("\nExpected:")
print("  fake_action → Upper Left (Low Reality, High Narrative)")
print("  true_action → Upper Right (High Reality, High Narrative)")
print("  silent_action → Lower Right (High Reality, Low Narrative)")
print("  honest_no_action → Lower Left (Low Reality, Low Narrative)")

# Check if fake_action is primarily in upper left
if 'fake_action' in category_stats:
    fake_mask = categories == 'fake_action'
    fake_upper_left = np.sum((reality_scores[fake_mask] <= 0) & (narrative_scores[fake_mask] > 0))
    fake_total = np.sum(fake_mask)
    fake_upper_left_pct = fake_upper_left / fake_total * 100 if fake_total > 0 else 0
    print(f"\nObserved:")
    print(f"  fake_action in Upper Left: {fake_upper_left}/{fake_total} ({fake_upper_left_pct:.1f}%)")


### 9.4 Token-Level Analysis

Visualize actual tokens at each position to check for token-level noise or variation.


In [ ]:
# Analyze token distribution at each position
from collections import Counter

print("Token analysis at each position:")
print("="*60)

for position in config.extraction.positions:
    pos_samples = [s for s in dataset.samples if s.position == position and s.layer == 16]
    
    if len(pos_samples) == 0:
        continue
    
    # Get token strings
    tokens = [s.token_str for s in pos_samples if s.token_str is not None]
    
    print(f"\n{position}:")
    print(f"  Total samples: {len(pos_samples)}")
    print(f"  Samples with token_str: {len(tokens)}")
    
    if len(tokens) > 0:
        # Count unique tokens
        token_counts = Counter(tokens)
        print(f"  Unique tokens: {len(token_counts)}")
        
        # Show most common tokens
        print(f"  Most common tokens:")
        for token, count in token_counts.most_common(10):
            token_repr = repr(token) if token else "None"
            pct = count / len(tokens) * 100
            print(f"    {token_repr[:40]:40s} ({count:3d}, {pct:5.1f}%)")
        
        # Check for common patterns (newlines, spaces, punctuation)
        newline_tokens = sum(1 for t in tokens if t and '\n' in t)
        space_tokens = sum(1 for t in tokens if t and t.strip() == '')
        print(f"\n  Token patterns:")
        print(f"    Contains newline: {newline_tokens:3d} ({newline_tokens/len(tokens)*100:5.1f}%)")
        print(f"    Whitespace only:  {space_tokens:3d} ({space_tokens/len(tokens)*100:5.1f}%)")

# Compare before_tool tokens for tool_used=True vs False
print("\n" + "="*60)
print("before_tool Token Comparison: tool_used=True vs False")
print("="*60)

before_tool_samples = [s for s in dataset.samples if s.position == "before_tool" and s.layer == 16]
tool_tokens = [s.token_str for s in before_tool_samples if s.tool_used and s.token_str]
no_tool_tokens = [s.token_str for s in before_tool_samples if not s.tool_used and s.token_str]

print(f"\ntool_used=True:  {len(tool_tokens)} samples")
if tool_tokens:
    tool_counts = Counter(tool_tokens)
    print(f"  Unique tokens: {len(tool_counts)}")
    print(f"  Top tokens:")
    for token, count in tool_counts.most_common(5):
        print(f"    {repr(token)[:40]:40s} ({count})")

print(f"\ntool_used=False: {len(no_tool_tokens)} samples")
if no_tool_tokens:
    no_tool_counts = Counter(no_tool_tokens)
    print(f"  Unique tokens: {len(no_tool_counts)}")
    print(f"  Top tokens:")
    for token, count in no_tool_counts.most_common(5):
        print(f"    {repr(token)[:40]:40s} ({count})")


## 5. Analyze Fake Action Cases

**Critical test:** Does the probe correctly identify fake action episodes?

In [ ]:
# Analyze probe on fake action cases
fake_analysis = analyze_probe_on_category(
    reality_probe,
    probe_dataset,
    category="fake_action",
    label_type="reality",
)

print(f"\nFake Action Analysis:")
print(f"  N samples: {fake_analysis['n_samples']}")
print(f"  Metrics:")
print(fake_analysis['metrics'])

# Check alignment with ground truth
fake_probs = fake_analysis['probabilities']
fake_preds = fake_analysis['predictions']
fake_labels = fake_analysis['true_labels']  # Should all be 0 (tool not used)

# How many does probe correctly identify as "tool not used"?
correct_on_fakes = np.mean(fake_preds == 0)
mean_prob_tool_used = np.mean(fake_probs)

print(f"\n**Probe on Fake Actions:**")
print(f"  Correctly predicts 'no tool': {correct_on_fakes:.1%}")
print(f"  Mean P(tool_used): {mean_prob_tool_used:.3f}")

if correct_on_fakes > 0.95:
    print("  ✓ Probe aligns with reality, not narrative!")

In [ ]:
# Histogram of P(tool_used) on fake vs true cases
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Get predictions on true action cases for comparison
true_analysis = analyze_probe_on_category(
    reality_probe,
    probe_dataset,
    category="true_action",
    label_type="reality",
)

# Plot fake action probabilities
axes[0].hist(fake_probs, bins=20, alpha=0.7, color='red', edgecolor='black')
axes[0].axvline(x=0.5, color='k', linestyle='--', linewidth=1)
axes[0].set_xlabel('P(tool_used)')
axes[0].set_ylabel('Count')
axes[0].set_title(f'Fake Actions (should cluster near 0)\nMean = {mean_prob_tool_used:.3f}')
axes[0].set_xlim(0, 1)

# Plot true action probabilities
true_probs = true_analysis['probabilities']
mean_prob_true = np.mean(true_probs)
axes[1].hist(true_probs, bins=20, alpha=0.7, color='green', edgecolor='black')
axes[1].axvline(x=0.5, color='k', linestyle='--', linewidth=1)
axes[1].set_xlabel('P(tool_used)')
axes[1].set_ylabel('Count')
axes[1].set_title(f'True Actions (should cluster near 1)\nMean = {mean_prob_true:.3f}')
axes[1].set_xlim(0, 1)

plt.tight_layout()
plt.savefig(config.data.figures_dir / "figure3_fake_vs_true_probs.png", dpi=300, bbox_inches='tight')
plt.show()

## 6. Layer Analysis

Which layers encode action-grounding information?

In [ ]:
# Train probes at each layer (using mid_response position)
layer_accuracies = {}

for layer in config.extraction.layers:
    print(f"\nTraining probe at layer: {layer}")
    
    layer_dataset = dataset.filter_by_position("mid_response").filter_by_layer(layer)
    
    if len(layer_dataset) == 0:
        print(f"  No samples found for layer {layer}")
        continue
    
    probe, _, test_metrics = train_and_evaluate(
        layer_dataset,
        label_type="reality",
        random_state=42,
    )
    
    layer_accuracies[layer] = test_metrics.accuracy
    print(f"  Test accuracy: {test_metrics.accuracy:.1%}")

In [ ]:
# Visualize layer analysis
fig = plot_layer_analysis(
    layer_accuracies,
    title="Reality Probe Accuracy by Layer",
    save_path=config.data.figures_dir / "figure5_layer_accuracy",
)

plt.show()

# Best layer
best_layer = max(layer_accuracies.items(), key=lambda x: x[1])
print(f"\nBest layer: {best_layer[0]} (accuracy: {best_layer[1]:.1%})")

## 7. Probe Direction Analysis

Are reality and narrative probes learning the same representation?

In [ ]:
# Compare probe directions
from src.analysis.probes import compare_probes

comparison = compare_probes(reality_probe, narrative_probe, normalize=True)

print(f"\nReality vs Narrative Probe:")
print(f"  Cosine similarity: {comparison['cosine_similarity']:.3f}")
print(f"  L2 distance: {comparison['l2_distance']:.3f}")

if abs(comparison['cosine_similarity']) > 0.8:
    print("  → Probes learn similar directions (aligned)")
elif abs(comparison['cosine_similarity']) < 0.3:
    print("  → Probes learn different directions (independent representations)")
else:
    print("  → Probes partially aligned")

## 8. Visualizations

In [ ]:
# Get test data for reality probe
train_dataset, test_dataset = probe_dataset.train_test_split(test_size=0.2, random_state=42)
X_test, y_test = test_dataset.to_sklearn_format("reality")

# Confusion matrix
y_pred = reality_probe.predict(X_test)
fig = plot_confusion_matrix(
    y_test,
    y_pred,
    labels=["No Tool", "Tool Used"],
    title="Reality Probe Confusion Matrix",
    save_path=config.data.figures_dir / "reality_probe_confusion",
)
plt.show()

In [ ]:
# ROC curve
y_proba = reality_probe.predict_proba(X_test)[:, 1]
auc, fpr, tpr, thresholds = compute_roc_auc(y_test, y_proba)

fig = plot_roc_curve(
    fpr,
    tpr,
    auc,
    title="Reality Probe ROC Curve",
    save_path=config.data.figures_dir / "reality_probe_roc",
)
plt.show()

## Summary

In [ ]:
print("=" * 60)
print("PHASE 2 RESULTS: MECHANISTIC PROBES")
print("=" * 60)

print(f"\nReality Probe Performance:")
print(f"  Test Accuracy: {reality_test_metrics.accuracy:.1%}")
print(f"  ROC-AUC: {reality_test_metrics.roc_auc:.3f}")

print(f"\nPosition Analysis:")
for pos, acc in position_accuracies.items():
    print(f"  {pos}: {acc:.1%}")

print(f"\nFake Action Analysis:")
print(f"  Correct on fakes: {correct_on_fakes:.1%}")
print(f"  Mean P(tool_used) on fakes: {mean_prob_tool_used:.3f}")

print(f"\nProbe Direction Comparison:")
print(f"  Cosine similarity: {comparison['cosine_similarity']:.3f}")

print("\n✓ Phase 2 complete: Linear probe can detect ground truth")
print("=" * 60)

## Next Steps

→ **Notebook 03:** Test cross-tool generalization